In [3]:
import subprocess
import json

In [ ]:
def gen_sbatch(config_string):
    sbatch_string = f"""#!/bin/bash

    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=8
    #SBATCH --time=0:40:00
    #SBATCH --mem=32GB
    #SBATCH --gres=gpu:1
    #SBATCH --job-name=Diffeo
    #SBATCH --mail-type=END
    #SBATCH --mail-user=cm6627@nyu.edu
    #SBATCH --output=slurm/slurm_%j.out
    #SBATCH --error=slurm/slurm_%j.err
    
    module purge
    
    singularity exec --nv \
      --overlay /scratch/cm6627/diffeo_cnn/my_env/overlay-15GB-500K.ext3:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-val.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-test.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-train.sqf:ro \
      /scratch/work/public/singularity/cuda12.3.2-cudnn9.0.0-ubuntu-22.04.4.sif \
      /bin/bash -c "source /ext3/env.sh; python fit.py '{config_string}'" """
    return sbatch_string

In [22]:
config = {
    "dropout_rate": 0.2,
    "batch_size": 64,
    "epochs": 1500,
    "optimizer": {
        "name": "AdamW",
        "lr": 1e-6,
        "weight_decay": 2.0
    },
    "dataset": "resnet18_imagenet1k_train"
}
config_string = str(json.dumps(config)).replace('"', '\\"')

with open("submit.sbatch", "w") as f:
    f.write(sbatch_string)

In [ ]:
with open('config.json', "w") as f:
    json.dump(config, f, indent=4)

subprocess.run(["sbatch ./submit.sbatch"], shell=True)